<a href="https://colab.research.google.com/github/FazilovDev/coursework/blob/master/PhysioNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install torch

In [0]:
import torch
import random
import numpy as np
import pandas as pd
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [0]:
from google.colab import drive
path = '/content/drive/My Drive/dataset'

In [8]:
!ls '/content/drive/My Drive/dataset'

test  train


In [0]:
import torch.nn as nn
class PhysioNet(nn.Module):

    def __init__(self):
        super(PhysioNet, self).__init__()

        self.batch_norm0 = torch.nn.BatchNorm2d(1)
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, padding=1, stride=1)
        self.act1 = torch.nn.LeakyReLU()
        #self.batch_norm1 = torch.nn.BatchNorm2d(4)
        #self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = torch.nn.Linear(62*19 * 4, 50)
        self.act4 = torch.nn.LeakyReLU()
        
        self.fc2 = torch.nn.Linear(50, 20)
        self.act5 = torch.nn.LeakyReLU()
        
        self.fc3 = torch.nn.Linear(20, 1)
    
    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.conv1(x)
        x = self.act1(x)
        #x = self.batch_norm1(x)
        #x = self.pool1(x)

        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
        x = self.fc1(x)
        x = self.act4(x)
        x = self.fc2(x)
        x = self.act5(x)
        x = self.fc3(x)
        return x

In [19]:
from torchsummary import summary
physionet = PhysioNet()
summary(physionet, (1, 64, 21))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1            [-1, 1, 64, 21]               2
            Conv2d-2            [-1, 4, 62, 19]             104
         LeakyReLU-3            [-1, 4, 62, 19]               0
            Linear-4                   [-1, 50]         235,650
         LeakyReLU-5                   [-1, 50]               0
            Linear-6                   [-1, 20]           1,020
         LeakyReLU-7                   [-1, 20]               0
            Linear-8                    [-1, 1]              21
Total params: 236,797
Trainable params: 236,797
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.08
Params size (MB): 0.90
Estimated Total Size (MB): 0.99
----------------------------------------------------------------


In [0]:
Y_train = pd.read_csv("/content/drive/My Drive/dataset/train/y.csv", delimiter=";")
Y_train = np.array(Y_train.drop(["0.5010000000000012"],axis=1))


In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
physionet = physionet.to(device)
#for param in physionet.parameters():
#    param.requires_grad = False

In [0]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(physionet.parameters(), lr=1.0e-3)

In [23]:
x = np.array(pd.read_csv("/content/drive/My Drive/dataset/test/{}.csv".format(1), delimiter=";"))[:-1]
print(x.shape)
x = np.delete(x.reshape(64,24),23,1)
x = np.delete(x.reshape(64,23),22,1)
x = np.delete(x.reshape(64,22),21,1)
print(x.shape)

(64, 24)
(64, 21)


In [0]:
y_test = pd.read_csv("/content/drive/My Drive/dataset/test/y.csv", delimiter=";")
y_test = np.array(y_test.drop(["0.7650000000001"],axis=1))
X_test = []
for i in range(len(y_test)):
  x = pd.read_csv("/content/drive/My Drive/dataset/test/{}.csv".format(i), delimiter=";")
  x = np.array(x)[:-1]
  x = np.delete(x.reshape(64,24),23,1)
  x = np.delete(x.reshape(64,23),22,1)
  x = np.delete(x.reshape(64,22),21,1)
  X_test.append(x)

In [25]:
test_accuracy_history = []
test_loss_history = []


len_x_train = len(Y_train)
batch_size = 100
for epoch in range(10000):
    order = np.random.permutation(len_x_train)
    for start_index in range(0, len_x_train, batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        X_train = []
        for file in batch_indexes:
          x = np.array(pd.read_csv("/content/drive/My Drive/dataset/train/{}.csv".format(file), delimiter=";"))[:-1]
          x = np.delete(x.reshape(64,24),23,1)
          x = np.delete(x.reshape(64,23),22,1)
          x = np.delete(x.reshape(64,22),21,1)
          X_train.append(x)
        #X_train = torch.DoubleTensor(np.array(pd.read_csv("/content/drive/My Drive/dataset/train/{}.csv".format(1), delimiter=";"))[:-1])
        preds = physionet.forward(torch.tensor(X_train).unsqueeze(1).float())
        y_batch = []
        for file in batch_indexes:
          y_batch.append(Y_train[file])

        loss_value = loss(preds.float(), torch.tensor(y_batch).float())
        loss_value.backward()
        
        optimizer.step()
    
    test_preds = physionet.forward(torch.tensor(X_test).unsqueeze(1).float())
    print(test_preds)
    test_loss_history.append(loss(test_preds.float(), torch.tensor(y_test).float()))#.data.cpu())
    
    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data.cpu()
    test_accuracy_history.append(accuracy)
    
    print(accuracy)

KeyboardInterrupt: ignored

In [0]:

X_train = torch.FloatTensor(np.array(pd.read_csv("/content/drive/My Drive/dataset/train/{}.csv".format(1), delimiter=";")))

In [0]:
X_train = []
for file in batch_indexes:
  x = np.double(np.array(pd.read_csv("/content/drive/My Drive/dataset/train/{}.csv".format(file), delimiter=";"), dtype=np.double)[:-1])
  X_train.append(x)


In [0]:
100*1364*1200 *20